# Случайные деревья
Идея: проверить значимость переменных "забил предыдущий", "забил два предыдущих" и тд

In [15]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier # случайный лес
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score # AUC для оценки качества модели
from sklearn.metrics import confusion_matrix # для определения FP, FN, TP, TN
from sklearn.model_selection import train_test_split 

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
raw = pd.read_csv('curry1415.csv', header=0)

In [3]:
# Интересно, есть ли разница для алгоритма между переменными "координаты" и "расстояние" + "угол"?
# добавим более точную переменную расстояния
raw['dist'] = np.sqrt(raw['x']**2 + raw['y']**2)

# введем угол броска

loc_x_zero = raw['x'] == 0
raw['angle'] = np.array([0]*len(raw))
raw['angle'][~loc_x_zero] = np.arctan(raw['y'][~loc_x_zero] / raw['x'][~loc_x_zero])
raw['angle'][loc_x_zero] = np.pi / 2 

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
# единая переменная оставшегося времени
raw['remaining_time'] = raw['minutes_remaining'] * 60 + raw['seconds_remaining']

In [6]:
# дропаем ненужные переменные
todrop = ['name', 'team_name', 'game_date', 'season', 'espn_player_id', 'team_id', \
          'espn_game_id', 'minutes_remaining', 'seconds_remaining', 'shot_distance', 'x', 'y', 'defender_name']

for i in todrop:
    raw = raw.drop(i, 1)

In [7]:
# создаем дамми-переменные из категориальных
categorical_vars = ['action_type', 'shot_type', 'opponent', 'period']

for i in categorical_vars:
    raw = pd.concat([raw, pd.get_dummies(raw[i], prefix=i)], 1)
    raw = raw.drop(i, 1)

In [8]:
# делим выборку на объясняющие переменные и таргетируемую
train = raw.drop('shot_made_flag', 1)
train_y = raw['shot_made_flag']

## Рандомное разделение выборки
just for fun попробуем рандомно разделить выборку и  прогнать алгоритм случайного леса. 

In [9]:
X_train, X_test, y_train, y_test = train_test_split(train, train_y, test_size=0.3, random_state=0)

In [40]:
rfc = RandomForestClassifier(n_estimators=150, max_depth=20)
rfc.fit(X_train, y_train)
pred=rfc.predict(X_test)

In [41]:
# Найдем самые простые оценки качества модели
a=confusion_matrix(y_test, pred)
acc=(a[0][0]+a[1][1])/len(y_test)
pr=a[1][1]/(a[1][1]+a[0][1])
rc=a[1][1]/(a[1][1]+a[1][0])
fmera=2*pr*rc/(pr+rc)
print(acc, pr, rc, fmera)

0.606741573034 0.66028708134 0.498194945848 0.567901234568


Результаты не очень хорошие. Алгоритм плохо предсказывает действительность

## Оптимизация параметров леса с k-fold кросс-валидацией

In [33]:
scores_n = []
range_n = [10, 50, 100, 150, 250, 500]
kf = KFold(n_splits=10)
for n in range_n:    
    rfc = RandomForestClassifier(n_estimators=n)
    for train_k, test_k in kf.split(train):
        rfc.fit(train.iloc[train_k], train_y.iloc[train_k])
        pred = rfc.predict(train.iloc[test_k])
        a=confusion_matrix(train_y[test_k], pred)
        pr=a[1][1]/(a[1][1]+a[0][1])
        rc=a[1][1]/(a[1][1]+a[1][0])
        fmera=2*pr*rc/(pr+rc)
    scores_n.append(fmera)
print(scores_n)

[0.55913978494623651, 0.5490196078431373, 0.61032863849765251, 0.56481481481481488, 0.56221198156682028, 0.59259259259259256]


Таким образом, судя по F-мере, лучшее число деревьев равно 100

In [37]:
scores_m = []
range_m = [2,4,8,10,20,50]
kf = KFold(n_splits=10)
for m in range_m:
    rfc = RandomForestClassifier(max_depth=m, n_estimators=100)
    for train_k, test_k in kf.split(train):
        rfc.fit(train.iloc[train_k], train_y.iloc[train_k])
        pred = rfc.predict(train.iloc[test_k])
        a=confusion_matrix(train_y[test_k], pred)
        pr=a[1][1]/(a[1][1]+a[0][1])
        rc=a[1][1]/(a[1][1]+a[1][0])
        fmera=2*pr*rc/(pr+rc)
    scores_m.append(fmera)
print(scores_m)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: RuntimeWarning: invalid value encountered in long_scalars
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: RuntimeWarning: invalid value encountered in long_scalars
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: RuntimeWarning: invalid value encountered in long_scalars
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: RuntimeWarning: invalid value encountered in long_scalars
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: RuntimeWarning: invalid value encountered in long_scalars
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: RuntimeWarning: invalid value encountered in long_scalars
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: RuntimeWarning: invalid value encountered in long_scalars


[0.32116788321167883, 0.39759036144578319, 0.56603773584905659, 0.57142857142857151, 0.61682242990654212, 0.56872037914691942]


Судя по F-мере, лучший параметр максимальной глубины деревьев равен 20